#### THIS NOTEBOOK IS SEPARATED BY SECTION FOR EACH PART OF THE MODEL ESTIMATION AND BACKTEST. READ EACH SECTION DESCRIPTION 

### This cell installs all required package in the virtual or global environment such that any user can run this notebook

In [30]:
import sys
!{sys.executable} -m pip install pandas numpy scikit-learn matplotlib scipy shap seaborn

import warnings
warnings.filterwarnings('ignore')


[notice] A new release of pip is available: 23.3.1 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


### This cell installs imports all required packages and functions from other files

In [31]:
import os


import Functions
import importlib
import Functions.class_data_processing as data
import Functions.classes_signals as signals
import Functions.classes_ml_models as ml_models
import Functions.class_descriptive_stats as descriptive_stats
import Functions.evaluation_ml_models_utils as evaluation
from Functions.other_functions import *

import pandas as pd
import numpy as np
from sklearn.metrics import r2_score, confusion_matrix
from sklearn.decomposition import PCA
from matplotlib import pyplot as plt
import seaborn as sns

importlib.reload(data)
importlib.reload(signals)
importlib.reload(Functions)
importlib.reload(ml_models)
importlib.reload(descriptive_stats)
importlib.reload(evaluation)

from Functions.class_data_processing import DataReader, SignalProcessor
from Functions.classes_signals import Return, MovingAverage, CrossMovingAverage, BollingerBands, MinMaxBreakout
from Functions.classes_ml_models import RidgeRegression, MultipleLinearRegression, LassoRegression, RandomForestClassification, KMeansClustering
from Functions.class_descriptive_stats import DescriptiveStatistics
from Functions.evaluation_ml_models_utils import plot_mean_importance, display_average_r2, compute_classification_metrics, plot_average_confusion_matrix
from sklearn.metrics import confusion_matrix

output_folder = "Outputs"


### We load all of our datasets and set the dates for out data splits

In [32]:
# Asset class file paths and sheet names
COMMOD_filepath = os.path.join('Data', 'Commodities.xlsx')
EQUITY_filepath = os.path.join('Data', 'Equities.xlsx')
FX_filepath = os.path.join('Data', 'FX.xlsx')
BONDS_filepath = os.path.join('Data', 'Bonds.xlsx')
MACRO_filepath = os.path.join('Data', 'macro_data.xlsx')


In [33]:
ASSET_CLASSES = ['COMMOD', 'EQUITY', 'FX', 'BONDS']
COMMOD_sheet_names = ['Crude Oil', 'Nat Gas', 'Copper', 'Wheat', 'Aluminium', 'Nickel', 'Gold', 'Silver', 'Corn', 'Cocoa', 'Soybean', 'Cattle']
EQUITY_sheet_names = ["SPY", "UK", "EUROSTOXX", "JAPAN", "HONK KONG", "FRANCE", "GERMANY", "CANADA", "RUSSELL2000", "QQQ", "EU", "TAIWAN", "ITALY", "SWITZERLAND", "SOUTH AFRICA"]
FX_sheet_names = ["USDJPY", "EURUSD", "GBPUSD","USDCAD", "NZDUSD", "USDMXN", "EURCHF", "NOKSEK", "EURGBP", "USDAUD"]
BONDS_sheet_names = ["US GOV", "CA GOV", "EURO GOV", "EM GOV", "US IG", "EURO IG", "CA IG", "US HY", "EURO HY"]

# Start and end dates for training, validation and testing sets
train_start_date = '2001-01-01'
train_end_date = '2012-12-31'
val_start_date = '2013-01-01'
val_end_date = '2014-12-31'
test_start_date = '2015-01-01'
test_end_date = '2024-08-31'

# Initialize dictionaries to store data instances and sheet names
asset_class_filepaths = {
    'COMMOD': COMMOD_filepath,
    'EQUITY': EQUITY_filepath,
    'FX': FX_filepath,
    'BONDS': BONDS_filepath
}

asset_class_sheet_names = {
    'COMMOD': COMMOD_sheet_names,
    'EQUITY': EQUITY_sheet_names,
    'FX': FX_sheet_names,
    'BONDS': BONDS_sheet_names
}


### We calculate signals for each asset in each asset class. The signals are calculated in the classes_signals.py file in Functions

In [34]:
# Initialize dictionaries to store predictions, models and covariance matrices
predictions_ridge = {}
models_ridge = {}

predictions_linreg = {}
models_linreg = {}

predictions_lasso = {}
models_lasso = {}

predictions_rf = {}
models_rf = {}

cov_matrices = {}
exp_ret_matrices = {}

train_asset_class_returns = {}

# Load data and signals for each asset class
for asset_class in ASSET_CLASSES:
    # Load data
    filepath = asset_class_filepaths[asset_class]
    sheet_names = asset_class_sheet_names[asset_class]
    
    # Create data instance for the current asset class
    data_instance = DataReader(filepath)
    data_instance.load_data(sheet_names)
    data_instance.data.columns = sheet_names
    data_instance.data = data_instance.data.groupby(data_instance.data.index).first().sort_index()
    
    # Store data instance in globals for access later
    globals()[f"{asset_class}_data_instance"] = data_instance
    globals()[f"{asset_class}_sheet_names"] = sheet_names

    # Initialize SignalProcessor for the current asset class from class
    signal_processor = SignalProcessor(data_instance.data)
    
    # Define the signal classes to run
    signal_classes = [Return, MovingAverage, CrossMovingAverage, BollingerBands, MinMaxBreakout]
    
    # Initialize signals dictionary to store signals for each asset in the asset class
    signals_dict = {}

    # set the train data
    train_data = data_instance.data.loc[train_start_date:train_end_date]
    
    # Calculate the overall return for the current asset class
    equal_weights = np.array([1/len(sheet_names)]*len(sheet_names))
    train_data_mth = train_data.resample('M').mean()
    returns = train_data.pct_change().dropna()*12
    train_asset_class_returns[asset_class] = equal_weights @ returns.T

    # Calculate the covariance matrix for the current asset class
    cov = PortfolioInputs(train_data)
    cov_matrices[asset_class] = cov.calculate_covariance_matrix()

    
    # Process signals for each asset in the current asset class
    for asset in sheet_names:
        # Define signal parameters
        signal_params = {
            'Return': ([22, 65], asset),
            'MovingAverage': ([22, 65], asset), 
            'CrossMovingAverage': ([(22, 65), (65, 260)], asset),
            'BollingerBands': ([22], asset),
            'MinMaxBreakout': ([65], asset)
        }
    
        asset_signals_df = signal_processor.process_signals(asset, signal_classes, signal_params)
        
        # Store the DataFrame in the signals dictionary
        signals_dict[asset] = asset_signals_df
    
    globals()[f"{asset_class}_signals_dict"] = signals_dict

### We fit our linear models (Ridge, Lasso and OLS regressions) and compute monthly predictions and store them. Only Ridge is used after.

In [35]:
# Initialize dictionaries to store aggregated feature importances for Ridge Regression
aggregated_feature_importances_ridge = {asset_class: [] for asset_class in ASSET_CLASSES}
ridge_r2_scores = {asset_class: [] for asset_class in ASSET_CLASSES}

# Now process each asset to train models and make predictions
for asset_class in ASSET_CLASSES:
    # Initialize predictions and models dictionary for this asset class
    predictions_ridge[asset_class] = {}
    models_ridge[asset_class] = {}
    predictions_lasso[asset_class] = {}
    models_lasso[asset_class] = {}
    models_linreg[asset_class] = {}
    predictions_linreg[asset_class] = {}
    
    # Access data and signals
    data_instance = globals()[f"{asset_class}_data_instance"]
    signals_dict = globals()[f"{asset_class}_signals_dict"]
    sheet_names = globals()[f"{asset_class}_sheet_names"]
    
    for asset in sheet_names:
        try:
            # Prepare training data
            train_data = data_instance.data[asset].loc[train_start_date:train_end_date]
            train_signals = signals_dict[asset].loc[train_start_date:train_end_date]
            
            # Ensure data is not empty
            if train_data.empty or train_signals.empty:
                print(f"Training data or signals for {asset} in {asset_class} is empty. Skipping.")
                continue
            
            # =====================
            # Ridge Regression
            # =====================
            
            # Initialize RidgeRegression model from class
            ridge_model = RidgeRegression(train_data, train_signals)
            
            # Define hyperparameter grid for Ridge Regression (alphas)
            ridge_alphas = np.linspace(0.0, 3000, 100)
            
            # Evaluate Ridge model to find best alpha
            best_alpha_ridge, best_r2_ridge = ridge_model.evaluate(n_splits=5, alphas=ridge_alphas)
            print(f"{asset_class} - {asset} (Ridge): Best alpha: {best_alpha_ridge}, Best R2: {best_r2_ridge}")
            
            # The fit method is already called inside evaluate()
            # Store the Ridge model
            models_ridge[asset_class][asset] = ridge_model

            # Store the R² score for Ridge
            ridge_r2_scores[asset_class].append(best_r2_ridge)

            # Collect feature importances from Ridge Regression
            feature_importance_df_ridge = ridge_model.get_feature_importances()
            aggregated_feature_importances_ridge[asset_class].append(feature_importance_df_ridge['Importance'].values)
            
            # =====================
            # Lasso Regression
            # =====================
            
            # Initialize LassoRegression model
            lasso_model = LassoRegression(train_data, train_signals)
            
            # Define hyperparameter grid for Lasso Regression (alphas)
            lasso_alphas = np.linspace(0.0, 3000, 100)
            
            # Evaluate Lasso model to find best alpha
            best_alpha_lasso, best_r2_lasso = lasso_model.evaluate(n_splits=5, alphas=lasso_alphas)
            print(f"{asset_class} - {asset} (Lasso): Best alpha: {best_alpha_lasso}, Best R2: {best_r2_lasso}")
            
            # The fit method is already called inside evaluate()
            # Store the Lasso model
            models_lasso[asset_class][asset] = lasso_model
            
            # =====================
            # Linear Regression
            # =====================

            model_linreg = MultipleLinearRegression(train_data, train_signals)
            model_linreg.fit()
            models_linreg[asset_class][asset] = model_linreg

                        # Prepare testing data
            test_signals_daily = signals_dict[asset].loc[test_start_date:test_end_date]
            test_data_daily = data_instance.data[asset].loc[test_start_date:test_end_date]

            # Ensure testing data is not empty
            if test_data_daily.empty or test_signals_daily.empty:
                print(f"Testing data or signals for {asset} in {asset_class} is empty. Skipping.")
                continue

            # Resample to monthly frequency
            test_signals_monthly = test_signals_daily.resample('M').last()
            test_data_monthly = test_data_daily.resample('M').last()

            # Calculate monthly returns for testing set (target variable)
            test_y = test_data_monthly.pct_change().shift(-1).dropna()

            # Align signals and target variable
            test_X = test_signals_monthly.loc[test_y.index]

            # Drop any remaining NaN values
            test_X = test_X.dropna()
            test_y = test_y.loc[test_X.index]

            # Ensure testing features and target are not empty
            if test_X.empty or test_y.empty:
                print(f"No overlapping data between testing features and target for {asset} in {asset_class}. Skipping.")
                continue

            # =====================
            # Ridge Predictions
            # =====================

            # Scale testing features using the scaler fitted on training data (Ridge)
            test_X_scaled_ridge = ridge_model.scaler.transform(test_X.values)

            # Predict on testing set using the trained Ridge model
            test_predictions_ridge = ridge_model.model.predict(test_X_scaled_ridge)

            # Create a DataFrame to store actual and predicted returns (Ridge)
            predictions_df_ridge = pd.DataFrame({
                'Actual Returns': test_y.values.flatten(),
                'Predicted Returns': test_predictions_ridge
            }, index=test_y.index)

            # Store Ridge predictions
            predictions_ridge[asset_class][asset] = predictions_df_ridge

            # =====================
            # Linear Regression Predictions
            # =====================

            # Scale testing features using the scaler fitted on training data (Linear Regression)
            test_X_scaled_linreg = model_linreg.scaler.transform(test_X.values)

            # Predict on testing set using the trained Linear Regression model
            test_predictions_linreg = model_linreg.model.predict(test_X_scaled_linreg)

            # Create a DataFrame to store actual and predicted returns (Linear Regression)
            predictions_df_linreg = pd.DataFrame({
                'Actual Returns': test_y.values.flatten(),
                'Predicted Returns': test_predictions_linreg
            }, index=test_y.index)

            # Store Linear Regression predictions
            predictions_linreg[asset_class][asset] = predictions_df_linreg

            # =====================
            # Lasso Predictions
            # =====================

            # Scale testing features using the scaler fitted on training data (Lasso)
            test_X_scaled_lasso = lasso_model.scaler.transform(test_X.values)

            # Predict on testing set using the trained Lasso model
            test_predictions_lasso = lasso_model.model.predict(test_X_scaled_lasso)

            # Create a DataFrame to store actual and predicted returns (Lasso)
            predictions_df_lasso = pd.DataFrame({
                'Actual Returns': test_y.values.flatten(),
                'Predicted Returns': test_predictions_lasso
            }, index=test_y.index)

            # Store Lasso predictions
            predictions_lasso[asset_class][asset] = predictions_df_lasso

        except Exception as e:
                print(f"An error occurred for {asset} in {asset_class}: {e}")
                continue

            # At this point, 'predictions_ridge' dictionary contains predictions for all assets across all asset classes




COMMOD - Crude Oil (Ridge): Best alpha: 3000.0, Best R2: -0.044399990031798245
COMMOD - Crude Oil (Lasso): Best alpha: 30.303030303030305, Best R2: -0.044635838826559214
COMMOD - Nat Gas (Ridge): Best alpha: 181.8181818181818, Best R2: -0.01997020388521469
COMMOD - Nat Gas (Lasso): Best alpha: 30.303030303030305, Best R2: -0.021955521191660176
COMMOD - Copper (Ridge): Best alpha: 969.6969696969697, Best R2: -0.1563449535757166
COMMOD - Copper (Lasso): Best alpha: 30.303030303030305, Best R2: -0.1600088082417407
COMMOD - Wheat (Ridge): Best alpha: 3000.0, Best R2: -0.016260457493367665
COMMOD - Wheat (Lasso): Best alpha: 30.303030303030305, Best R2: -0.015229282297428837
COMMOD - Aluminium (Ridge): Best alpha: 3000.0, Best R2: -0.1483038702796585
COMMOD - Aluminium (Lasso): Best alpha: 30.303030303030305, Best R2: -0.14262870214794718
COMMOD - Nickel (Ridge): Best alpha: 3000.0, Best R2: -0.1983765696876319
COMMOD - Nickel (Lasso): Best alpha: 30.303030303030305, Best R2: -0.19566467313

In [36]:
feature_names_ridge = ridge_model.X.columns
positions = [2,3]
names = []
for col_name in feature_names_ridge:
    name = remove_words_at_positions(col_name, positions)
    names.append(name) 
feature_names_ridge = names


### Compute average feature importance and Out of Sample R2 for Ridge models across all asset classes to evaluate model fit and accuracy. 

In [37]:
for asset_class in ASSET_CLASSES:
     # Plot feature importances and R2 for Ridge
     plot_mean_importance(
         asset_class,
         aggregated_feature_importances_ridge[asset_class],
         feature_names_ridge,
         model_name="Ridge Regression"
     )
     # Display average R2 for Ridge for each asset class
     display_average_r2(
         asset_class,
         ridge_r2_scores[asset_class],
         model_name="Ridge Regression"
     )


Average Feature Importances for COMMOD (Ridge Regression):
                Feature  Mean Importance
2        Signal EMA 22D         0.002088
6  Signal Bollinger 22D        -0.000121
5  Signal MACD 65D 260D        -0.000252
0     Signal Return 22D        -0.000447
3        Signal EMA 65D        -0.000613
7   Signal Breakout 65D        -0.001415
1     Signal Return 65D        -0.002223
4   Signal MACD 22D 65D        -0.002451
Plot saved to: Outputs\COMMOD_Ridge Regression_importance.png
Average R² for Ridge Regression in COMMOD: -0.0488

Average Feature Importances for EQUITY (Ridge Regression):
                Feature  Mean Importance
0     Signal Return 22D         0.002244
2        Signal EMA 22D         0.001296
3        Signal EMA 65D         0.000394
5  Signal MACD 65D 260D         0.000361
1     Signal Return 65D         0.000325
7   Signal Breakout 65D        -0.001017
4   Signal MACD 22D 65D        -0.002097
6  Signal Bollinger 22D        -0.002142
Plot saved to: Outputs\EQUITY

### We fit our Random Forest model and compute monthly predictions (probability of up movement) and store them.

In [38]:
# Initialize dictionaries to store aggregated feature importances and confusion matrices
aggregated_feature_importances = {asset_class: [] for asset_class in ASSET_CLASSES}
confusion_matrices_rf = {asset_class: [] for asset_class in ASSET_CLASSES}

# Process each asset to train models and make predictions
for asset_class in ASSET_CLASSES:
    # Initialize dictionaries for predictions and models for each asset class
    predictions_rf[asset_class] = {}
    models_rf[asset_class] = {}
    
    # Access data and signals
    data_instance = globals()[f"{asset_class}_data_instance"]
    signals_dict = globals()[f"{asset_class}_signals_dict"]
    sheet_names = globals()[f"{asset_class}_sheet_names"]
    
    for asset in sheet_names:
        try:
            # Prepare training data
            train_data = data_instance.data[asset].loc[train_start_date:train_end_date]
            train_signals = signals_dict[asset].loc[train_start_date:train_end_date]
            
            # Skip if training data or signals are empty
            if train_data.empty or train_signals.empty:
                print(f"Training data or signals for {asset} in {asset_class} is empty. Skipping.")
                continue
            
            # Initialize and evaluate RandomForest model
            rf_model = RandomForestClassification(train_data, train_signals)
            param_grid = {
                'n_estimators': [50, 100, 200],
                'max_depth': [None, 5, 10],
                'min_samples_split': [2, 5],
                'min_samples_leaf': [1, 2],
                'class_weight': [None, 'balanced']
            }
            best_params, best_score = rf_model.evaluate(n_splits=5, param_grid=param_grid, scoring='neg_log_loss')
            print(f"{asset_class} - {asset}: Best Params: {best_params}, Best Negative Log Loss: {best_score}")
            
            # Store the fitted model
            models_rf[asset_class][asset] = rf_model
            
            # Prepare testing data
            test_signals_daily = signals_dict[asset].loc[test_start_date:test_end_date]
            test_data_daily = data_instance.data[asset].loc[test_start_date:test_end_date]
            if test_data_daily.empty or test_signals_daily.empty:
                print(f"Testing data or signals for {asset} in {asset_class} is empty. Skipping.")
                continue
            
            # Resample to monthly frequency and align target and features
            test_signals_monthly = test_signals_daily.resample('M').last()
            test_data_monthly = test_data_daily.resample('M').last()
            test_y = test_data_monthly.pct_change().shift(-1).dropna()
            test_y_binary = (test_y > 0).astype(int)
            test_X = test_signals_monthly.loc[test_y_binary.index].dropna()
            test_y_binary = test_y_binary.loc[test_X.index]
            test_y = test_y.loc[test_X.index]  # Align actual returns

            # Skip if testing data is empty after alignment
            if test_X.empty or test_y_binary.empty:
                print(f"No overlapping data between testing features and target for {asset} in {asset_class}. Skipping.")
                continue
            
            # Scale testing features and predict
            test_X_scaled = rf_model.scaler.transform(test_X.values)
            test_predictions_binary = rf_model.model.predict(test_X_scaled)
            
            # Compute and store confusion matrix
            cm = confusion_matrix(test_y_binary, test_predictions_binary)
            confusion_matrices_rf[asset_class].append(cm)
            
            # Store probability predictions
            proba_class_1 = rf_model.model.predict_proba(test_X_scaled)[:, 1]
            predictions_rf[asset_class][asset] = pd.DataFrame({
                'Actual return': test_y.values.flatten(),
                'Proba of going up': proba_class_1
            }, index=test_y.index)

            # Collect feature importances
            aggregated_feature_importances[asset_class].append(rf_model.get_feature_importances()['Importance'].values)

        except Exception as e:
            print(f"An error occurred for {asset} in {asset_class}: {e}")
            continue

# At this point, 'predictions_rf' dictionary contains predictions for all assets across all asset classes


COMMOD - Crude Oil: Best Params: {'class_weight': None, 'max_depth': 5, 'min_samples_leaf': 2, 'min_samples_split': 5, 'n_estimators': 100}, Best Negative Log Loss: -0.7347529839576883
COMMOD - Nat Gas: Best Params: {'class_weight': 'balanced', 'max_depth': 5, 'min_samples_leaf': 2, 'min_samples_split': 5, 'n_estimators': 200}, Best Negative Log Loss: -0.755398486663344
COMMOD - Copper: Best Params: {'class_weight': 'balanced', 'max_depth': 5, 'min_samples_leaf': 1, 'min_samples_split': 5, 'n_estimators': 50}, Best Negative Log Loss: -0.7449309367572894
COMMOD - Wheat: Best Params: {'class_weight': 'balanced', 'max_depth': 5, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 200}, Best Negative Log Loss: -0.7185787084357923
COMMOD - Aluminium: Best Params: {'class_weight': 'balanced', 'max_depth': 5, 'min_samples_leaf': 2, 'min_samples_split': 5, 'n_estimators': 200}, Best Negative Log Loss: -0.753518561017563
COMMOD - Nickel: Best Params: {'class_weight': 'balanced', 'max

In [39]:
feature_names = rf_model.X.columns
positions = [2,3]
names = []
for col_name in feature_names:
    name = remove_words_at_positions(col_name, positions)
    names.append(name) 

feature_names = names


### Compute average feature importance and Out of Sample Confusion matric for Random Forest model across all asset classes to evaluate model fit and accuracy. 

In [40]:
# Compute average feature importances per asset class and display
for asset_class in ASSET_CLASSES:
    # Plot feature importances for Random Forest
    plot_mean_importance(
        asset_class,
        aggregated_feature_importances[asset_class],
        feature_names,
        model_name="Random Forest"
    )

    # Compute and display classification metrics for Random Forest
    compute_classification_metrics(
        asset_class,
        confusion_matrices_rf[asset_class]
    )

    # Plot average confusion matrix for Random Forest
    plot_average_confusion_matrix(
        asset_class,
        confusion_matrices_rf[asset_class]
    )


Average Feature Importances for COMMOD (Random Forest):
                Feature  Mean Importance
0     Signal Return 22D         0.195511
1     Signal Return 65D         0.177832
2        Signal EMA 22D         0.157884
3        Signal EMA 65D         0.149158
4   Signal MACD 22D 65D         0.141671
5  Signal MACD 65D 260D         0.132829
6  Signal Bollinger 22D         0.043005
7   Signal Breakout 65D         0.002110
Plot saved to: Outputs\COMMOD_Random Forest_importance.png

Metrics for COMMOD:
Average Precision: 0.55
Average Recall: 0.63
Average Accuracy: 0.54
Average F1 Score: 0.58

Average Confusion Matrix for COMMOD:
[[0.44560358 0.55439642]
 [0.37235543 0.62764457]]
Plot saved to: Outputs\COMMOD_average_confusion_matrix.png

Average Feature Importances for EQUITY (Random Forest):
                Feature  Mean Importance
0     Signal Return 22D         0.193892
1     Signal Return 65D         0.173885
2        Signal EMA 22D         0.161568
3        Signal EMA 65D         0.

### For each model, we compute cross sectional z-scores of the predictions across all assets in an asset class to measure relative trend

In [41]:
# Initialize dictionaries to store z-scores for each asset class
ridge_zscores = {}
linreg_zscores = {}
rf_zscores = {}

# Loop through each asset class
for asset_class in ASSET_CLASSES:
    # Extract predictions for the asset class
    ridge_predictions = predictions_ridge[asset_class]  # Ridge predictions
    linreg_predictions = predictions_linreg[asset_class]  # Linear Regression predictions
    rf_predictions = predictions_rf[asset_class]  # Random Forest predictions
    
    # Initialize DataFrame to store z-scores for the asset class
    ridge_zscores[asset_class] = pd.DataFrame()
    linreg_zscores[asset_class] = pd.DataFrame()
    rf_zscores[asset_class] = pd.DataFrame()

    # Combine predictions from all assets in the asset class into a single DataFrame for Ridge
    ridge_combined_df = pd.concat([ridge_predictions[asset]['Predicted Returns'] for asset in asset_class_sheet_names[asset_class]], axis=1)
    ridge_combined_df.columns = asset_class_sheet_names[asset_class]  # Set columns as asset names
    
    # Combine predictions from all assets in the asset class into a single DataFrame for Linear Regression
    linreg_combined_df = pd.concat([linreg_predictions[asset]['Predicted Returns'] for asset in asset_class_sheet_names[asset_class]], axis=1)
    linreg_combined_df.columns = asset_class_sheet_names[asset_class]  # Set columns as asset names

    # Combine predictions from all assets in the asset class into a single DataFrame for RF
    rf_combined_df = pd.concat([rf_predictions[asset]['Proba of going up'] for asset in asset_class_sheet_names[asset_class]], axis=1)
    rf_combined_df.columns = asset_class_sheet_names[asset_class]  # Set columns as asset names
    
    # Compute z-scores for Ridge predictions
    ridge_zscores[asset_class] = (ridge_combined_df.subtract(ridge_combined_df.mean(axis=1), axis=0)
                                  .divide(ridge_combined_df.std(axis=1), axis=0))
    
    # Compute z-scores for Linear Regression predictions
    linreg_zscores[asset_class] = (linreg_combined_df.subtract(linreg_combined_df.mean(axis=1), axis=0)
                                   .divide(linreg_combined_df.std(axis=1), axis=0))
    
    # Compute z-scores for Random Forest predictions
    rf_zscores[asset_class] = (rf_combined_df.subtract(rf_combined_df.mean(axis=1), axis=0)
                               .divide(rf_combined_df.std(axis=1), axis=0))

print(rf_zscores['FX'])


              USDJPY    EURUSD    GBPUSD    USDCAD    NZDUSD    USDMXN  \
Date                                                                     
2015-01-31 -0.576963 -1.030499  0.551436 -1.514868  1.066373  0.479412   
2015-02-28 -0.039668  0.820521 -0.370880 -0.898851 -1.386154  1.250761   
2015-03-31 -0.633894 -0.268382  1.686045 -0.885632 -0.879995  0.898807   
2015-04-30  0.178183  0.916322  1.343684 -0.588286  0.246514 -0.108786   
2015-05-31  1.706249  0.331570 -0.829710 -0.885219  1.742118 -0.666143   
...              ...       ...       ...       ...       ...       ...   
2024-03-31 -0.191432 -0.361858  1.938385 -1.584788 -0.333860  0.657769   
2024-04-30  0.811563  0.158720  1.495869  0.071642 -0.708263  1.117229   
2024-05-31  0.928008  1.141540 -1.555278  0.421059 -1.293176 -0.306109   
2024-06-30  2.092131 -0.037107  0.014444 -1.397179  0.704017  0.788587   
2024-07-31  0.295194  0.431514 -0.933498 -0.197737  0.503024  1.390788   

              EURCHF    NOKSEK    EUR

### We aggregate the z-scores of both models for each asset in each asset class

In [42]:
# Initialize dictionary to store the summed z-scores for each asset class
summed_zscores = {}

# Loop through each asset class
for asset_class in ASSET_CLASSES:
    # Extract z-score DataFrames for Ridge and Random Forest
    ridge_zscores_df = ridge_zscores[asset_class]
    linreg_zscores_df = linreg_zscores[asset_class]
    rf_zscores_df = rf_zscores[asset_class]
    
    # Ensure both DataFrames have the same structure (same columns and index)
    ridge_zscores_df = ridge_zscores_df.loc[rf_zscores_df.index, rf_zscores_df.columns]
    linreg_zscores_df = linreg_zscores_df.loc[rf_zscores_df.index, rf_zscores_df.columns]
    
    # Sum the z-scores from both Ridge and Random Forest models
    summed_zscores[asset_class] = ridge_zscores_df  + rf_zscores_df

summed_zscores['FX']

,USDJPY,EURUSD,GBPUSD,USDCAD,NZDUSD,USDMXN,EURCHF,NOKSEK,EURGBP,USDAUD
Date,,,,,,,,,,
2015-01-31,-0.988896,-0.693938,0.385977,-3.612833,1.663186,0.608161,3.221416,0.127118,1.040946,-1.751137
2015-02-28,-0.983131,1.473926,-0.312209,-1.136207,-0.138064,1.960642,-1.463853,-1.329960,3.043899,-1.115041
2015-03-31,-1.508825,0.819067,1.466136,-2.333819,0.507275,1.593827,-0.180691,0.412381,1.867942,-2.643293
2015-04-30,-0.651235,1.769703,1.325230,1.374044,1.108692,0.037516,-0.891374,-2.355512,-1.278853,-0.438212
2015-05-31,1.261138,0.976281,-0.498005,-3.049164,3.191713,-0.021557,-0.552216,0.397969,0.032744,-1.738902
...,...,...,...,...,...,...,...,...,...,...
2024-03-31,-0.713027,0.431870,2.223252,-2.383574,1.257801,1.211012,-1.893427,1.419393,0.231827,-1.785127
2024-04-30,0.165400,0.960598,1.637611,-0.077901,0.695454,1.864951,-2.920834,-1.087600,1.148318,-2.385996
2024-05-31,0.363650,1.962081,-1.211515,-1.290555,0.194943,0.347538,0.213676,-0.926968,1.504765,-1.157616


### We transform the aggregate z-scores into rankings. Higher z-score gets higher ranking, meaning asset has better return prospects. 

In [43]:
assets_rankings = {}

for asset_class in ASSET_CLASSES:

    # Get z-scores for each method
    summed_zscores_df = summed_zscores[asset_class]

    # Rank each asset based on its z-score at each time t
    assets_rankings[asset_class] = summed_zscores_df.rank(axis=1, ascending=False)

assets_rankings['EQUITY']

,SPY,UK,EUROSTOXX,JAPAN,HONK KONG,FRANCE,GERMANY,CANADA,RUSSELL2000,QQQ,EU,TAIWAN,ITALY,SWITZERLAND,SOUTH AFRICA
Date,,,,,,,,,,,,,,,
2015-01-31,14.0,13.0,3.0,5.0,7.0,10.0,4.0,6.0,12.0,11.0,2.0,9.0,8.0,15.0,1.0
2015-02-28,8.0,15.0,4.0,7.0,6.0,14.0,2.0,11.0,10.0,9.0,12.0,5.0,13.0,3.0,1.0
2015-03-31,13.0,7.0,3.0,6.0,10.0,9.0,2.0,12.0,8.0,11.0,4.0,5.0,14.0,1.0,15.0
2015-04-30,12.0,10.0,6.0,8.0,4.0,9.0,15.0,3.0,7.0,14.0,11.0,5.0,13.0,2.0,1.0
2015-05-31,13.0,10.0,8.0,7.0,6.0,9.0,3.0,4.0,15.0,14.0,5.0,12.0,11.0,1.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-03-31,5.0,15.0,11.0,13.0,7.0,10.0,4.0,2.0,9.0,14.0,6.0,3.0,12.0,8.0,1.0
2024-04-30,11.0,9.0,4.0,15.0,5.0,12.0,7.0,3.0,13.0,8.0,2.0,10.0,14.0,6.0,1.0
2024-05-31,7.0,12.0,6.0,13.0,4.0,2.0,5.0,9.0,8.0,14.0,3.0,10.0,11.0,15.0,1.0


### MACRO CLUSTERING - We load the macro data

In [44]:
macro = DataReader(MACRO_filepath)
econ = macro.load_data(["ECON"])
price = macro.load_data(["PRATE"])

econ.index = pd.to_datetime(econ.index)
price.index = pd.to_datetime(price.index)
econ = econ.loc[econ.index.intersection(price.index)]
price = price.loc[econ.index.intersection(price.index)]
macro_data = pd.concat([econ, price], axis=1)
macro_data.dropna(inplace=True)
macro_data

,RGDP YoY,CPI YoY,UNEM,FED FUNDS,SP500 Vol,RATE 10Y
Date,,,,,,
1954-07-01,-0.76833,-0.01242,6.0,1.03,0.078188,2.30
1954-10-01,2.72862,-0.49585,5.3,0.99,0.034480,2.43
1955-01-01,6.17020,-0.59362,4.7,1.34,0.039658,2.61
1955-04-01,7.78024,-0.56980,4.4,1.50,0.053287,2.75
1955-07-01,8.01592,-0.23597,4.1,1.94,0.077818,2.90
...,...,...,...,...,...,...
2022-07-01,2.29639,8.28869,3.5,2.19,0.110215,2.90
2022-10-01,1.31633,7.09080,3.6,3.65,0.140244,3.98
2023-01-01,2.28081,5.74983,3.5,4.52,0.143436,3.53


### Set the 4 indicators we will use

In [45]:
macro_data["Yield Curve"] = macro_data["RATE 10Y"] - macro_data["FED FUNDS"]
cols_to_drop = ["RATE 10Y", "FED FUNDS", "SP500 Vol"]
macro_data.drop(cols_to_drop, axis=1, inplace=True)
macro_data

,RGDP YoY,CPI YoY,UNEM,Yield Curve
Date,,,,
1954-07-01,-0.76833,-0.01242,6.0,1.27
1954-10-01,2.72862,-0.49585,5.3,1.44
1955-01-01,6.17020,-0.59362,4.7,1.27
1955-04-01,7.78024,-0.56980,4.4,1.25
1955-07-01,8.01592,-0.23597,4.1,0.96
...,...,...,...,...
2022-07-01,2.29639,8.28869,3.5,0.71
2022-10-01,1.31633,7.09080,3.6,0.33
2023-01-01,2.28081,5.74983,3.5,-0.99


### Set a training, validation and test set for the k-means clustering algortithm + transform the data into time series Z-scores

In [46]:
macro_train_start_date = '1948-01-01'
macro_train_end_date = '2012-12-31'
macro_val_start_date = '2013-01-01'
macro_val_end_date = '2014-12-31'
macro_test_start_date = '2015-01-01'
macro_test_end_date = '2024-07-01'
macro_data_train = macro_data.loc[macro_train_start_date:macro_train_end_date]
macro_data_val = macro_data.loc[macro_val_start_date:macro_val_end_date]
macro_data_test = macro_data.loc[macro_test_start_date:macro_test_end_date]

macro_data_train_z = (macro_data_train - macro_data_train.mean()) / macro_data_train.std()
macro_data_train_z


,RGDP YoY,CPI YoY,UNEM,Yield Curve
Date,,,,
1954-07-01,-1.605925,-1.321707,0.009893,0.166877
1954-10-01,-0.178490,-1.490565,-0.428073,0.271155
1955-01-01,1.226344,-1.524715,-0.803472,0.166877
1955-04-01,1.883553,-1.516395,-0.991172,0.154609
1955-07-01,1.979756,-1.399791,-1.178872,-0.023278
...,...,...,...,...
2011-10-01,-0.662420,-0.149087,1.636624,0.663733
2012-01-01,-0.215452,-0.329115,1.448924,0.534918
2012-04-01,-0.311267,-0.657937,1.386357,0.553320


### We estimate the k-means algorithm on the training data and store the clusters

In [47]:
model = KMeansClustering(macro_data_train_z)
kmeans_model = model.k_means_clustering(4)

clusters = kmeans_model.labels_
centroids = kmeans_model.cluster_centers_
macro_data_train_z['Cluster'] = clusters

### We compute PCAs to be able to visualize the estimated clusters in 2D. Not important for the strategy.

In [48]:
# Perform PCA on the macro data
pca = PCA(n_components=2)
principal_components = pca.fit_transform(macro_data_train_z.drop(columns='Cluster'))

# Project centroids into PCA space
centroids_pca = pca.transform(centroids)

pca_df = pd.DataFrame(data=principal_components, columns=['PC1', 'PC2'])
pca_df['Cluster'] = clusters


plt.figure(figsize=(12, 8))
for cluster in pca_df['Cluster'].unique():
    cluster_data = pca_df[pca_df['Cluster'] == cluster]
    plt.scatter(cluster_data['PC1'], cluster_data['PC2'], label=f'Cluster {cluster}')

plt.scatter(
    centroids_pca[:, 0], 
    centroids_pca[:, 1], 
    s=250, c='black', marker='X', label='Centroids'
)

plt.title('K-means Clustering Visualization')
plt.xlabel('Principal Component 1')
plt.ylabel('Principal Component 2')
plt.legend()
plt.grid(True)

filename = "kmeans_clustering_pca_visualization.png"
filepath = os.path.join(output_folder, filename)
plt.savefig(filepath, bbox_inches='tight')
plt.close() 


### We calculate the means of our macro indicators to differentiate economic regimes.

In [49]:
macro_data_train['Cluster'] = clusters
cluster_means = macro_data_train.groupby('Cluster').mean()
cluster_means


,RGDP YoY,CPI YoY,UNEM,Yield Curve
Cluster,,,,
0,4.128136,3.444206,6.522857,2.224714
1,3.985715,2.794678,4.674757,0.320583
2,0.169548,3.591809,8.134884,2.138605
3,1.890482,11.063913,6.244444,-2.621667


#### We see that Cluster 0 is a high growth and upward sloping yield curve environment. 
#### Cluster 1 is decent growth, but flat yield curve. 
#### Cluster 2 is recessions. 
#### Cluster 3 is High inflation and iverted yield curve environment. 

### Based on these 4 disctinct economic regimes, we establish discretionary asset class deviations based on expected returns founded on financial and economic theory.

In [50]:
# Define the asset classes
ASSET_CLASSES = ['EQUITY', 'BONDS', 'FX', 'COMMOD']

# Define the base equal weights
base_weights = {asset_class: 1 / len(ASSET_CLASSES) for asset_class in ASSET_CLASSES}

# Define the adjustment rules based on clusters
cluster_adjustments = {
    0: {'EQUITY': +0.10, 'BONDS': -0.10, 'FX': 0.00, 'COMMOD': 0.00},
    1: {'EQUITY': +0.05, 'BONDS': -0.05, 'FX': +0.05, 'COMMOD': -0.05},
    2: {'EQUITY': -0.10, 'BONDS': +0.10, 'FX': -0.05, 'COMMOD': +0.05},
    3: {'EQUITY': +0.05, 'BONDS': -0.10, 'FX': -0.05, 'COMMOD': +0.10},
}


### We now predict clusters in the test set

In [51]:
# Standardize the macro data in the test set using the training mean and std
macro_data_train = macro_data_train.drop(columns='Cluster')
# Standardize the macro data in the test set using the training mean and std
macro_data_test_z = (macro_data_test - macro_data_train.mean()) / macro_data_train.std()

# Predict clusters on the test set
test_clusters = kmeans_model.predict(macro_data_test_z)
macro_data_test_z['Cluster'] = test_clusters

# Assign the cluster labels to the dates in the test set
cluster_series = pd.Series(test_clusters, index=macro_data_test_z.index)



### Based on predicted clusters in the test set, we compute asset class weights over the whole backtesting period. 

In [52]:
# Initialize a DataFrame to store adjusted weights over time
dates = macro_data_test_z.index
adjusted_weights_df = pd.DataFrame(index=dates, columns=ASSET_CLASSES)

for date in dates:
    cluster = cluster_series.loc[date]
    adjustments = cluster_adjustments.get(cluster, {asset_class: 0.0 for asset_class in ASSET_CLASSES})
    
    # Adjust the base weights
    adjusted_weights = {}
    for asset_class in ASSET_CLASSES:
        adjusted_weight = base_weights[asset_class] + adjustments[asset_class]
        adjusted_weights[asset_class] = adjusted_weight
    
    # Normalize the weights to ensure they sum to 1
    total_weight = sum(adjusted_weights.values())
    normalized_weights = {asset_class: weight / total_weight for asset_class, weight in adjusted_weights.items()}
    
    # Store the normalized weights
    adjusted_weights_df.loc[date] = normalized_weights


In [53]:
adjusted_weights_df

,EQUITY,BONDS,FX,COMMOD
Date,,,,
2015-01-01,0.35,0.15,0.25,0.25
2015-04-01,0.3,0.2,0.3,0.2
2015-07-01,0.35,0.15,0.25,0.25
2015-10-01,0.3,0.2,0.3,0.2
2016-01-01,0.3,0.2,0.3,0.2
2016-04-01,0.3,0.2,0.3,0.2
2016-07-01,0.3,0.2,0.3,0.2
2016-10-01,0.3,0.2,0.3,0.2
2017-01-01,0.3,0.2,0.3,0.2


## BACKTEST
### We now compute the overall backtest of the strategy over the testing set.
#### We compute 3 strategies. The first is our overall strategy with the macro deviations over the asset classes and the ranking methodology inside the asset classes. The second is the strategy without the macro overlay, which means we simply use equal weights over the asset class portfolios. The final one is the benchmark, which is not only equal weight over the asset classes but also within the assets in each class. This means the benchmark doesnt use the ranks from our models nor the macro deviations.

In [54]:
# Step 1: Initialize Asset Rankings
assets_rankings = {}

for asset_class in ASSET_CLASSES:
    # Get summed z-scores DataFrame for the asset class
    summed_zscores_df = summed_zscores[asset_class]
    
    # Rank each asset based on its summed z-score at each time t
    assets_rankings[asset_class] = summed_zscores_df.rank(axis=1, ascending=False)

# Initialize dictionaries to store cumulative returns, individual returns, and weights
strategy_cum_returns = {}
asset_class_returns = {}
asset_weights = {}

# Define testing date range
test_start_date = macro_test_start_date
test_end_date = macro_test_end_date

# Step 2: Calculate strategy-based weights for each asset class and compute returns
for asset_class in ASSET_CLASSES:
    data_instance = globals()[f"{asset_class}_data_instance"]
    test_data_daily = data_instance.data.loc[test_start_date:test_end_date]
    
    # Resample to monthly frequency and calculate monthly returns
    test_data_monthly = test_data_daily.resample('M').last()
    test_data_monthly = test_data_monthly.pct_change().shift(-1).dropna()
    
    assets_rankings_df = assets_rankings[asset_class]
    
    # Ensure indices match
    common_index = test_data_monthly.index.intersection(assets_rankings_df.index)
    test_data_monthly = test_data_monthly.loc[common_index]
    assets_rankings_df = assets_rankings_df.loc[common_index]
    
    # Initialize DataFrame to store strategy weights
    strategy_weights = pd.DataFrame(index=assets_rankings_df.index, columns=assets_rankings_df.columns)
    
    # Calculate strategy weights based on inverse ranks
    for i in range(len(assets_rankings_df)):
        # Get the ranks of assets for the current date
        ranked_assets = assets_rankings_df.iloc[i]
        
        # Calculate inverse rank weights (higher rank gets higher weight)
        inverse_ranks = 1 / ranked_assets
        
        # Normalize the weights so that they sum up to 1
        normalized_weights = inverse_ranks / inverse_ranks.sum()
        
        # Assign weights to the DataFrame for the current row
        strategy_weights.iloc[i] = normalized_weights.values
    
    # Store the asset weights for this asset class
    asset_weights[asset_class] = strategy_weights
    
    # Calculate monthly portfolio returns using strategy weights
    monthly_returns = (test_data_monthly * strategy_weights).sum(axis=1)
    asset_class_returns[asset_class] = monthly_returns
    
    # Calculate cumulative returns for the strategy-based portfolio
    strategy_cum_returns[asset_class] = (1 + monthly_returns).cumprod()

# Step 3: Compute asset class volatilities using covariance matrices and inverse ranking weights
asset_class_vols = {}

for asset_class in ASSET_CLASSES:
    # Get the covariance matrix for the asset class
    cov_matrix = cov_matrices[asset_class]
    
    # Get the strategy weights for the asset class
    strategy_weights = asset_weights[asset_class]
    
    # Ensure the covariance matrix and weights have matching columns
    cov_matrix = cov_matrix.loc[strategy_weights.columns, strategy_weights.columns]
    
    # Compute average weights over the period
    avg_weights = strategy_weights.iloc[0]
    
    # Convert to numpy array
    weights = avg_weights.values.astype(float)
    
    # Compute portfolio variance: w.T * Cov * w
    portfolio_var = np.dot(weights.T, np.dot(cov_matrix, weights))
    
    # Take square root to get volatility
    portfolio_vol = np.sqrt(portfolio_var)
    
    # Store the volatility for the asset class
    asset_class_vols[asset_class] = portfolio_vol

# Step 4: Compute risk parity weights for asset classes based on calculated volatilities
inverse_vols = {asset_class: 1 / vol for asset_class, vol in asset_class_vols.items()}
total_inverse_vols = sum(inverse_vols.values())
risk_parity_weights = {asset_class: inverse_vol / total_inverse_vols for asset_class, inverse_vol in inverse_vols.items()}

# Step 5: Calculate global strategy portfolio returns using risk parity weights
global_portfolio_returns = pd.Series(0, index=next(iter(asset_class_returns.values())).index)
adjusted_weights_df.index = adjusted_weights_df.index.to_period('M').to_timestamp('M')

global_portfolio_returns_cluster = pd.Series(0, index=next(iter(asset_class_returns.values())).index)

for asset_class, returns in asset_class_returns.items():
    returns_rp = returns.reindex(global_portfolio_returns.index).fillna(0)
    global_portfolio_returns += risk_parity_weights[asset_class] * returns_rp
    
    weights_cluster = adjusted_weights_df[asset_class].astype(float)
    weights_cluster = weights_cluster.reindex(returns.index).fillna(method='ffill')
    
    common_index = weights_cluster.index.intersection(returns.index)
    weights_cluster = weights_cluster.loc[common_index]
    returns_cluster = returns.loc[common_index]
    
    weighted_returns = weights_cluster * returns_cluster
    global_portfolio_returns_cluster = global_portfolio_returns_cluster.add(weighted_returns, fill_value=0)

global_strategy_cum_returns = (1 + global_portfolio_returns).cumprod()
global_strategy_cum_returns_cluster = (1 + global_portfolio_returns_cluster).cumprod()

# -------------------------
# Benchmark Calculation
# -------------------------

equal_weight_returns_dict = {}
equal_weight_cum_returns = {}
equal_weight_asset_class_vols = {}

for asset_class in ASSET_CLASSES:
    data_instance = globals()[f"{asset_class}_data_instance"]
    test_data_daily = data_instance.data.loc[test_start_date:test_end_date]
    test_data_monthly = test_data_daily.resample('M').last()
    test_data_monthly = test_data_monthly.pct_change().shift(-1).dropna()
    
    num_assets = len(test_data_monthly.columns)
    equal_weights = np.ones(num_assets) / num_assets
    
    test_data_monthly = test_data_monthly.loc[test_data_monthly.index.intersection(global_portfolio_returns.index)]
    
    equal_weight_returns = test_data_monthly.dot(equal_weights)
    equal_weight_returns_dict[asset_class] = equal_weight_returns
    
    cum_returns = (1 + equal_weight_returns).cumprod()
    equal_weight_cum_returns[asset_class] = cum_returns
    
    cov_matrix = cov_matrices[asset_class]
    cov_matrix = cov_matrix.loc[test_data_monthly.columns, test_data_monthly.columns]
    
    portfolio_var = np.dot(equal_weights.T, np.dot(cov_matrix, equal_weights))
    portfolio_vol = np.sqrt(portfolio_var)
    equal_weight_asset_class_vols[asset_class] = portfolio_vol

inverse_vols_benchmark = {asset_class: 1 / vol if vol != 0 else 0 for asset_class, vol in equal_weight_asset_class_vols.items()}
total_inverse_vols_benchmark = sum(inverse_vols_benchmark.values())
risk_parity_weights_benchmark = {asset_class: inverse_vol / total_inverse_vols_benchmark if total_inverse_vols_benchmark != 0 else 1 / len(ASSET_CLASSES) for asset_class, inverse_vol in inverse_vols_benchmark.items()}

benchmark_portfolio_returns = pd.Series(0, index=global_portfolio_returns.index)

for asset_class, returns in equal_weight_returns_dict.items():
    returns_benchmark = returns.reindex(benchmark_portfolio_returns.index).fillna(0)
    benchmark_portfolio_returns += risk_parity_weights_benchmark[asset_class] * returns_benchmark

benchmark_strategy_cum_returns = (1 + benchmark_portfolio_returns).cumprod()




In [55]:
# -------------------------
# Plotting
# -------------------------

# Ensure all cumulative return series have the same index
common_index = global_strategy_cum_returns.index.intersection(global_strategy_cum_returns_cluster.index).intersection(benchmark_strategy_cum_returns.index)
global_strategy_cum_returns = global_strategy_cum_returns.loc[common_index]
global_strategy_cum_returns_cluster = global_strategy_cum_returns_cluster.loc[common_index]
benchmark_strategy_cum_returns = benchmark_strategy_cum_returns.loc[common_index]

# Plot 1: Individual Asset Classes' Cumulative Returns (Equal Weight)
plt.figure(figsize=(14, 7))
for asset_class, cum_returns in equal_weight_cum_returns.items():
    plt.plot(cum_returns.index, cum_returns, label=f'{asset_class} Equal Weight', linestyle='--')

plt.title('Cumulative Returns of Individual Asset Classes (Equal Weight)')
plt.xlabel('Date')
plt.ylabel('Cumulative Returns')
plt.legend()
plt.grid(True)

filename = "individual_asset_classes_equal_weight.png"
filepath = os.path.join(output_folder, filename)
plt.savefig(filepath, bbox_inches='tight')
plt.close()

# Plot 2: Portfolio Performance Comparison
plt.figure(figsize=(14, 7))
plt.plot(global_strategy_cum_returns, label='Strat sans macro', linewidth=2, linestyle=':')
plt.plot(global_strategy_cum_returns_cluster, label='Macro Clustering Portfolio', linewidth=2)
plt.plot(benchmark_strategy_cum_returns, label='Benchmark Portfolio', linewidth=2, linestyle='-.')
plt.title('Portfolio Performance Comparison')
plt.xlabel('Date')
plt.ylabel('Cumulative Returns')
plt.legend()
plt.grid(True)

filename = "portfolio_performance_comparison.png"
filepath = os.path.join(output_folder, filename)
plt.savefig(filepath, bbox_inches='tight')
plt.close()

# Plot 3: Return Differences Between Portfolios
plt.figure(figsize=(14, 7))

# Calculate return differences
return_diff_global_benchmark = global_strategy_cum_returns - benchmark_strategy_cum_returns
return_diff_cluster_benchmark = global_strategy_cum_returns_cluster - benchmark_strategy_cum_returns
return_diff_cluster_global = global_strategy_cum_returns_cluster - global_strategy_cum_returns

# Plot the return differences
plt.plot(return_diff_global_benchmark, label='Strat sans macro - Benchmark', linewidth=2)
plt.plot(return_diff_cluster_benchmark, label='Macro Clustering - Benchmark', linewidth=2)
plt.plot(return_diff_cluster_global, label='Macro Clustering - Strat sans macro', linewidth=2)
plt.title('Return Differences Between Portfolios')
plt.xlabel('Date')
plt.ylabel('Cumulative Return Difference')
plt.legend()
plt.grid(True)

filename = "return_differences_between_portfolios.png"
filepath = os.path.join(output_folder, filename)
plt.savefig(filepath, bbox_inches='tight')
plt.close()



### We compute descriptive statistics over the strategies on the backtest returns.

In [56]:
stats = DescriptiveStatistics(global_portfolio_returns_cluster)
stats.performance_summary()

,Value
Sharpe Ratio,0.163378
Max Drawdown,-0.136201
Cumulative Return,0.528875
Mean Return,0.048409
Mean Volatility,0.085535


In [57]:
stats = DescriptiveStatistics(global_portfolio_returns)
stats.performance_summary()


,Value
Sharpe Ratio,0.148663
Max Drawdown,-0.096435
Cumulative Return,0.235831
Mean Return,0.023329
Mean Volatility,0.045301


In [58]:
stats = DescriptiveStatistics(benchmark_portfolio_returns)
stats.performance_summary()

,Value
Sharpe Ratio,0.180234
Max Drawdown,-0.080398
Cumulative Return,0.187194
Mean Return,0.018513
Mean Volatility,0.029651
